In [1]:
try:
    import google.colab
    google.colab.drive.mount('/content/drive/')
    %cd ../content/drive/Othercomputers/Laptop/Flood_Detection
except:
    pass

In [2]:
import sys
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics
from mlpath import mlquest as mlq
import numpy as np
sys.path.append('../../')
from DataPreparation.DataPreparation import read_data
from sklearn.metrics import classification_report
sys.path.append('../')
from SimpleFeatures.SimpleFeatures import *

In [3]:
mlq.start_quest('LBP-QDA', table_dest='../../', log_defs=True)

### Reading and Preprocessing

In [4]:
x_train_d, x_val_d, y_train_d, y_val_d = mlq.l(read_data)(gray=True,saved=True)

x_train shape:  (737, 256, 256)
x_val shape:  (185, 256, 256)


### Feature Extraction: LBP Features

In [5]:
x_train_f = mlq.l(get_lbp)(x_train_d,radius=4)
x_val_f= mlq.l(get_lbp)(x_val_d,radius=4)

In [6]:
print(x_train_f.shape)

(737, 65536)


### Dimensionality Reduction: PCA

In [7]:
pca,x_train_f=mlq.l(apply_pca)(x_train_f,n_components=512,pca_obj=None)
x_val_f=mlq.l(apply_pca)(x_val_f,n_components=512,pca_obj=pca)

In [8]:
print(x_train_f.shape)

(737, 512)


### Initiating the Model

In [9]:
model = mlq.l(QuadraticDiscriminantAnalysis)(reg_param=0.0)

### Training the model

In [10]:
model.fit(x_train_f, y_train_d)

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

### Validate the model

In [11]:
F1 = metrics.f1_score(y_val_d, model.predict(x_val_f), average='macro').item()
Accuracy = metrics.accuracy_score(y_val_d, model.predict(x_val_f)).item()
mlq.log_metrics(Accuracy, F1)
report = classification_report(y_val_d, model.predict(x_val_f))
print(report)

              precision    recall  f1-score   support

           0       0.58      0.75      0.65        87
           1       0.69      0.51      0.59        98

    accuracy                           0.62       185
   macro avg       0.63      0.63      0.62       185
weighted avg       0.64      0.62      0.62       185



### Tracking

In [12]:
mlq.end_quest()
mlq.show_logs(quest_name='LBP-QDA', table_dest='../../', last_k=10)